In [ ]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import  tqdm_notebook

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
ls

gdrive/  sample_data/


In [ ]:
cd gdrive

/content/gdrive


In [ ]:
cd My Drive


/content/gdrive/My Drive


In [ ]:
cd cs 522

/content/gdrive/My Drive/cs 522


In [ ]:
cd project/project/our project

/content/gdrive/My Drive/cs 522/project/project/our project


In [ ]:
directory_path = '/content/gdrive/My Drive/cs 522/project/project/our project'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/cs 522/project/project/our project/lang_id_train_1.csv')

test_df = pd.read_csv('/content/gdrive/My Drive/cs 522/project/project/our project/lang_id_test.csv')

In [ ]:
ls

 bert-master/                     model_without_language_model.ckpt
 bert-master.zip                  pregenerate_training_data.py
'colal notebooks'/                README.md
 data_lm.txt                      sentence_embedding/
 eval.jsonlines                   sentence_embedding.zip
 finetuned_lm/                    simple_lm_finetuning.py
 finetune_on_pregenerated.py      test.jsonlines
 lang_id_eval.csv                 training/
 lang_id_test.csv                 train.jsonlines
 lang_id_train_1.csv              uncased_L-12_H-768_A-12/
 lang_id_train.csv                uncased_L-12_H-768_A-12.zip
 model_with_language_model.ckpt   Untitled.ipynb


In [ ]:
train_df_all = pd.read_csv(os.path.join(directory_path,'lang_id_train_all.csv'))
test_df_all = pd.read_csv(os.path.join(directory_path,'lang_id_test_all.csv'))

FileNotFoundError: ignored

In [ ]:
train_df.shape,test_df.shape

((6000, 3), (2000, 2))

In [ ]:
train_df.head(n=2)

,native_language,language_id,text
0,Japanese,0,"who, whom, whose I have often thinking that, I..."
1,Japanese,0,books I love reading. It's been one of my hobb...


In [ ]:
z= pd.read_csv('/content/gdrive/My Drive/cs 522/project/project/our project/lang_id_train_1.csv')


In [ ]:
z['native_language'].unique()

array(['Japanese', 'Korean', 'Vietnamese', 'Mandarin', 'Russian', 'Thai',
       'Spanish', 'Cantonese', 'Polish', 'Arabic'], dtype=object)

In [ ]:
ls

 bert-master/                  pregenerate_training_data.py
 bert-master.zip               README.md
'colal notebooks'/             sentence_embedding/
 data_lm.txt                   sentence_embedding.zip
 eval.jsonlines                simple_lm_finetuning.py
 finetuned_lm/                 test.jsonlines
 finetune_on_pregenerated.py   training/
 lang_id_eval.csv              train.jsonlines
 lang_id_test.csv              uncased_L-12_H-768_A-12/
 lang_id_train_1.csv           uncased_L-12_H-768_A-12.zip
 lang_id_train.csv             Untitled.ipynb


In [ ]:
train_df = train_df.drop(columns='native_language')


In [ ]:
test_df = test_df.drop(columns='native_language')


In [ ]:
test_df


,text
0,"got an official job offer Today, I decided to ..."
1,I am bum. I am bum.
2,exam... I will have to take the exam tomorrow....
3,Self-introduction and my situation I work in f...
4,Tough Schedule Hi there. So tough schedule las...
...,...
1995,friendship Why do we looking for make friends...
1996,"A Lecture Or A Torture Yesterday, I had this t..."
1997,...... I have been in New Zealand for seven mo...
1998,A messege to my favourite magazine Hi guys How...


In [ ]:
lm_df = pd.concat([train_df[['text']],test_df[['text']]])

In [ ]:
lm_df.shape

(8000, 1)

In [ ]:
tqdm.pandas()
changed_text=lm_df.text.apply(lambda x:x+"\n"+"\n")

In [ ]:
open(os.path.join(directory_path,'data_lm.txt'), "w").write(''.join(changed_text))

5665449

In [ ]:
!pip install pytorch_transformers
from pytorch_transformers import *

     |████████████████████████████████| 184kB 2.8MB/s 
     |████████████████████████████████| 890kB 7.6MB/s 
     |████████████████████████████████| 1.0MB 17.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=dce5ab13a134eadf59e8a683bdbfeb058ebe30bd2b06e422d25fd2ecd1842d0f
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [ ]:
%cd /content/drive/My Drive

[Errno 2] No such file or directory: '/content/drive/My Drive'
/content/gdrive/My Drive/cs 522/project/project/our project


In [ ]:
cd Colab\ Notebooks

In [ ]:
cd pytorch-transformers/examples/lm_finetuning

In [ ]:
import shutil

In [ ]:
src = directory_path+"/data_lm.txt" 
dst = os.getcwd()


In [ ]:
os.listdir(os.getcwd())

['lang_id_train.csv',
 'lang_id_test.csv',
 'README.md',
 'pregenerate_training_data.py',
 'simple_lm_finetuning.py',
 'finetune_on_pregenerated.py',
 'lang_id_eval.csv',
 'uncased_L-12_H-768_A-12.zip',
 'bert-master.zip',
 'sentence_embedding.zip',
 'Untitled.ipynb',
 'eval.jsonlines',
 'test.jsonlines',
 'train.jsonlines',
 'uncased_L-12_H-768_A-12',
 'sentence_embedding',
 '.ipynb_checkpoints',
 'bert-master',
 'training',
 'finetuned_lm',
 'colal notebooks',
 'lang_id_train_1.csv',
 'model_without_language_model.ckpt',
 'model_with_language_model.ckpt',
 'data_lm.txt']

In [ ]:
!python3 pregenerate_training_data.py --train_corpus data_lm.txt --bert_model bert-base-uncased --do_lower_case --output_dir training/ --epochs_to_generate 2 --max_seq_len 256

100% 231508/231508 [00:00<00:00, 319319.64B/s]
Loading Dataset: 16000 lines [00:14, 1081.15 lines/s]
Epoch:   0% 0/2 [00:00<?, ?it/s]
Document:   0% 0/8000 [00:00<?, ?it/s]
Document:   4% 326/8000 [00:00<00:02, 3253.28it/s]
Document:   8% 649/8000 [00:00<00:02, 3245.45it/s]
Document:  12% 947/8000 [00:00<00:02, 3159.24it/s]
Document:  16% 1252/8000 [00:00<00:02, 3125.15it/s]
Document:  19% 1558/8000 [00:00<00:02, 3102.83it/s]
Document:  23% 1874/8000 [00:00<00:01, 3118.75it/s]
Document:  27% 2179/8000 [00:00<00:01, 3094.74it/s]
Document:  31% 2467/8000 [00:00<00:01, 3024.47it/s]
Document:  35% 2771/8000 [00:00<00:01, 3027.84it/s]
Document:  38% 3065/8000 [00:01<00:01, 3000.75it/s]
Document:  42% 3367/8000 [00:01<00:01, 3004.38it/s]
Document:  46% 3680/8000 [00:01<00:01, 3038.46it/s]
Document:  50% 3995/8000 [00:01<00:01, 3070.01it/s]
Document:  54% 4300/8000 [00:01<00:01, 3034.12it/s]
Document:  58% 4602/8000 [00:01<00:01, 3017.12it/s]
Document:  62% 4928/8000 [00:01<00:00, 3085.30it/s

In [ ]:
os.listdir(os.getcwd())

['lang_id_train.csv',
 'lang_id_test.csv',
 'README.md',
 'pregenerate_training_data.py',
 'simple_lm_finetuning.py',
 'finetune_on_pregenerated.py',
 'lang_id_eval.csv',
 'uncased_L-12_H-768_A-12.zip',
 'bert-master.zip',
 'sentence_embedding.zip',
 'Untitled.ipynb',
 'eval.jsonlines',
 'test.jsonlines',
 'train.jsonlines',
 'uncased_L-12_H-768_A-12',
 'sentence_embedding',
 '.ipynb_checkpoints',
 'bert-master',
 'training',
 'finetuned_lm',
 'colal notebooks',
 'lang_id_train_1.csv',
 'model_without_language_model.ckpt',
 'model_with_language_model.ckpt',
 'data_lm.txt']

In [ ]:
!python3 finetune_on_pregenerated.py --pregenerated_data training/ --bert_model bert-base-uncased --do_lower_case --train_batch_size 16  --output_dir finetuned_lm/ --epochs 2

2020-04-29 22:51:17,751: device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
2020-04-29 22:51:17,756: Output directory (finetuned_lm) already exists and is not empty!
2020-04-29 22:51:18,506: loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2020-04-29 22:51:19,289: https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpwgtu4gom
100% 433/433 [00:00<00:00, 328118.09B/s]
2020-04-29 22:51:20,042: copying /tmp/tmpwgtu4gom to cache at /root/.cache/torch/pytorch_transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
2020-04-29 22:51:20,042: creating metadata fi

In [ ]:
np.random.seed(56)

In [ ]:
train_df.text = train_df.text.str.lower()
sentences = train_df.text.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]


In [ ]:
labels = train_df.language_id.values

In [ ]:
print(labels)

[0 0 0 ... 9 9 9]


In [ ]:
%%time
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[1])

Tokenize the first sentence:
['[CLS]', 'books', 'i', 'love', 'reading', '.', 'it', "'", 's', 'been', 'one', 'of', 'my', 'ho', '##bbies', 'since', 'i', 'was', 'little', '.', 'so', 'frustrating', 'that', 'i', 'can', "'", 't', 'read', 'an', 'english', 'book', 'as', 'quickly', 'as', 'a', 'japanese', 'book', '.', '(', 'a', 'harry', 'potter', 'book', 'took', 'me', '3', 'full', 'months', 'to', 'finish', '!', ')', 'but', 'i', 'just', 'can', "'", 't', 'stop', 'buying', 'books', '.', 'that', "'", 's', 'why', 'my', 'room', 'is', 'a', 'terrible', 'mess', '-', '-', '-', '-', 'un', '##rea', '##d', 'books', 'are', 'piled', 'up', 'here', 'and', 'there', '.', '[SEP]']
CPU times: user 11 s, sys: 57.4 ms, total: 11 s
Wall time: 11.8 s


In [ ]:
len(tokenized_texts[0])

43

In [ ]:
##uncomment all three lines below to convert to ids
input_ids=[]
for i in tqdm_notebook(range(len(tokenized_texts))):
   input_ids.append(tokenizer.convert_tokens_to_ids(tokenized_texts[i]))

Token indices sequence length is longer than the specified maximum sequence length for this model (607 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (704 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [ ]:
MAX_LEN = 256
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
input_ids.shape

(6000, 256)

In [ ]:
#Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
labels.shape

(6000,)

In [ ]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,random_state=56, test_size=0.2 ,stratify=labels)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=56, test_size=0.2,stratify=labels)

In [ ]:
train_labels.shape

(4800,)

In [ ]:
#Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [ ]:
batch_size = 10

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
## Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
##device ='cpu'
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=10)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
lr = 2e-5
max_grad_norm = 1.0
num_total_steps = 1000
num_warmup_steps = 100
warmup_proportion = float(num_warmup_steps) / float(num_total_steps)  # 0.1


### In PyTorch-Transformers, optimizer and schedules are splitted and instantiated like this:
optimizer = AdamW(model.parameters(), lr=lr, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=num_warmup_steps, t_total=num_total_steps)  # PyTorch scheduler

In [ ]:
len(train_dataloader)

480

In [ ]:
total_step = len(train_dataloader)

# Store our loss and accuracy for plotting
train_loss_set = []


epochs = 4

# trange is a tqdm wrapper around the normal python range
for epoch in tqdm_notebook(range(epochs)):
  
  

    # Training
    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for i, batch in enumerate(train_dataloader):
      # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
      loss = outputs[0]
      train_loss_set.append(loss.item())    
      # Backward pass
      loss.backward()
      # Update parameters and take a step using the computed gradient
      optimizer.step()
      scheduler.step()
      optimizer.zero_grad()
      if (i) % 50 == 0:
        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, epochs, i+1, total_step, loss.item()))
      

Epoch [1/4], Step [1/480], Loss: 2.3672
Epoch [1/4], Step [51/480], Loss: 2.3191
Epoch [1/4], Step [101/480], Loss: 2.1671
Epoch [1/4], Step [151/480], Loss: 2.5398
Epoch [1/4], Step [201/480], Loss: 1.5257
Epoch [1/4], Step [251/480], Loss: 1.2963
Epoch [1/4], Step [301/480], Loss: 1.9835
Epoch [1/4], Step [351/480], Loss: 1.9636
Epoch [1/4], Step [401/480], Loss: 0.9682
Epoch [1/4], Step [451/480], Loss: 1.7770
Epoch [2/4], Step [1/480], Loss: 1.1724
Epoch [2/4], Step [51/480], Loss: 1.1086
Epoch [2/4], Step [101/480], Loss: 0.8167
Epoch [2/4], Step [151/480], Loss: 1.3300
Epoch [2/4], Step [201/480], Loss: 1.1979
Epoch [2/4], Step [251/480], Loss: 1.3831
Epoch [2/4], Step [301/480], Loss: 0.9015
Epoch [2/4], Step [351/480], Loss: 1.0532
Epoch [2/4], Step [401/480], Loss: 0.9417
Epoch [2/4], Step [451/480], Loss: 1.1857
Epoch [3/4], Step [1/480], Loss: 0.6198
Epoch [3/4], Step [51/480], Loss: 1.0200
Epoch [3/4], Step [101/480], Loss: 1.0703
Epoch [3/4], Step [151/480], Loss: 0.8869
E

In [ ]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for i, batch in enumerate(validation_dataloader):
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask, b_labels = batch
      # Forward pass
      outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      # print (outputs)
      prediction = torch.argmax(outputs[0],dim=1)
      total += b_labels.size(0)
      correct+=(prediction==b_labels).sum().item()

In [ ]:
b_labels

tensor([4, 1, 9, 1, 0, 0, 2, 3, 2, 0], device='cuda:0')

In [ ]:
print('Test Accuracy of the model on val data is: {} %'.format(100 * correct / total)) 

Test Accuracy of the model on val data is: 55.333333333333336 %


using finetune model

In [ ]:
directory_path+"/pytorch-transformers/examples/lm_finetuning/finetuned_lm"

'/content/gdrive/My Drive/cs 522/project/project/our project/pytorch-transformers/examples/lm_finetuning/finetuned_lm'

In [ ]:
cd finetuned_lm/

[Errno 2] No such file or directory: 'finetuned_lm/'
/content/gdrive/My Drive/cs 522/project/project/our project/finetuned_lm


In [ ]:
ls

added_tokens.json  pytorch_model.bin        tokenizer_config.json
config.json        special_tokens_map.json  vocab.txt


In [ ]:
model_path= '/content/drive/My Drive/Colab Notebooks/pytorch-transformers/examples/lm_finetuning/finetuned_lm' ## model is stored at this directory.

In [ ]:
MAX_LEN = 128
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

this snippet helps to extract the fine tuned model

In [ ]:
## Define model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device ='cpu'
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=10)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
len(train_dataloader)

480

-----x_true---------y_pred

In [ ]:
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in train_dataloader:

  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 6,000 test sentences...
    DONE.


In [ ]:
import sklearn.metrics as skm
import numpy as np

true_labels_r=np.argmax(true_labels, axis=1)
##true_labels_r[3]

predictions_r=np.argmax(predictions, axis=1)
predictions_r=np.argmax(predictions_r, axis=1)
predictions_r[1]

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(true_labels_r, predictions_r)
cm

array([[12, 13,  4,  4,  7,  1,  6,  8,  7,  2],
       [ 8, 11,  8,  5,  7,  3,  5,  7,  3,  0],
       [16,  6,  7,  2,  3,  3,  4, 10,  1,  3],
       [ 5,  4,  3,  6,  2,  2, 11,  4,  8,  1],
       [10,  7,  5,  8,  2,  3,  4,  3,  4,  1],
       [16,  9,  7,  7,  1,  5,  7,  4,  4,  0],
       [11,  4,  5,  6,  7,  4,  6,  5,  4,  1],
       [ 9,  5,  1,  4,  4,  2,  2,  2,  5,  1],
       [ 3,  5,  2,  3,  4,  0,  3,  3,  6,  0],
       [ 8, 11,  2,  2,  2,  1,  4,  2,  2,  0]])

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(true_labels_r, predictions_r))

              precision    recall  f1-score   support

           0       0.12      0.19      0.15        64
           1       0.15      0.19      0.17        57
           2       0.16      0.13      0.14        55
           3       0.13      0.13      0.13        46
           4       0.05      0.04      0.05        47
           5       0.21      0.08      0.12        60
           6       0.12      0.11      0.11        53
           7       0.04      0.06      0.05        35
           8       0.14      0.21      0.16        29
           9       0.00      0.00      0.00        34

    accuracy                           0.12       480
   macro avg       0.11      0.11      0.11       480
weighted avg       0.12      0.12      0.11       480



In [ ]:
len(true_labels_r)

480

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
ax = sns.barplot(x=list(range(len(matthews_set))), y=matthews_set, ci=None)

plt.title('MCC Score per Batch')
plt.ylabel('MCC Score (-1 to +1)')
plt.xlabel('Batch #')

plt.show()

NameError: ignored

In [ ]:
# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)

NameError: ignored

In [ ]:
print('Test Accuracy of the model on val data is: {} %'.format(100 * correct / total))

NameError: ignored

In [ ]:
torch.save(model.state_dict(), directory_path+'/model_with_language_model.ckpt')

This snippet saves the entire model

In [ ]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = directory_path

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

this snippet will load the model back 

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassification.from_pretrained(output_dir)
tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

In [ ]:
labels.shape

NameError: ignored